Nota generada a partir de [liga](https://www.dropbox.com/s/5bc6tn39o0qqg35/1.3.Condicion_estabilidad_y_normas.pdf?dl=0)

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab-local -p 8888:8888 -d palmoreck/jupyterlab:1.1.0
```

Detener el contenedor de docker:

```
docker stop jupyterlab-local
```


# 1.3 Condición de un problema y estabilidad de un algoritmo

Dos temas fundamentales en el análisis numérico son: la **condición de un problema** y **estabilidad de un algoritmo**. El condicionamiento tiene que ver con el comportamiento de un problema ante perturbaciones y la estabilidad con el comportamiento de un algoritmo (usado para resolver un problema) ante perturbaciones.

La exactitud de un cálculo dependerá finalmente de una combinación de estos términos:

<p style="text-align: center;">Exactitud = Condición + Estabilidad</p>

La falta de exactitud en un problema se presenta entonces por problemas mal condicionados (no importando si los algoritmos son estables o inestables) y algoritmos inestables (no importando si los problemas son mal o bien condicionados).

## Perturbaciones

La condición de un problema y estabilidad de un algoritmo hacen referencia al término **perturbación**. Tal término conduce a pensar en perturbaciones "chicas" o "grandes". Para dar una medida de lo anterior se utiliza el concepto de **norma**. Ver final de esta nota para definición de norma y propiedades.

## Condición de un problema

---

## Estabilidad de un algoritmo

---

## Norma

Una norma define una medida de distancia en un conjunto y da nociones de tamaño, vecindad, convergencia y continuidad.

**Normas vectoriales**

Sea $\mathbb{R}^n$ el conjunto de $n$-tuplas o vectores columna o $1$-arreglo de orden $1$, esto es:

$$x \in \mathbb{R}^n \iff x = \left[\begin{array}{c}
    x_1\\
    x_2\\
    \vdots\\
    x_n
    \end{array} \right] \text{ con } x_i \in \mathbb{R}$$
 

Una norma vectorial en $\mathbb{R}^n$ es una función $g: \mathbb{R}^n \rightarrow \mathbb{R}$ que satisface las siguientes propiedades:

* $g$ es no negativa: $g(x) \geq 0 \forall x \in \mathbb{R}^n$.

* $g$ es definida: $g(x) = 0 \iff x = 0$.

* $g$ satisface la desigualdad del triángulo: $$g(x+y) \leq g(x) + g(y) \forall x,y \in \mathbb{R}^n.$$


* $g$ es homogénea: $g(\alpha x)=|\alpha|g(x), \forall \alpha \in \mathbb{R}, \forall x \in \mathbb{R}^n$.

Notación: $g(x) = ||x||$.

**Comentarios:**

* Una norma es una generalización del valor absoluto de $\mathbb{R}$: $|x|, x \in \mathbb{R}.$

* Un espacio vectorial con una norma definida en éste se le llama **espacio vectorial normado**.

* Una norma es una medida de la longitud de un vector.

* Con una norma es posible definir conceptos como distancia entre vectores: $x,y \in \mathbb{R}^n: \text{dist}(x,y) = ||x-y||$.

* Existen varias normas en $\mathbb{R}^n$ siendo las más comunes:

 * La norma $\mathcal{l}_2$, Euclidiana o norma $2$: $||x||_2$.
 
 * La norma $\mathcal{l}_1$ o norma $1$: $||x||_1$.
 
 * La norma $\infty$ o de Chebyshev o norma infinito: $||x||_\infty$.
 
 Las normas anteriores pertenecen a una familia parametrizada por una constante $p, p \geq 1$ cuyo nombre es norma $\mathcal{l}_p$:
 
 $$ ||x||_p = \left(\displaystyle \sum_{i=1}^n|x_i|^p \right )^{1/p}.$$
 


->La norma $\infty$ se encuentra en esta familia como límite: 

$$||x||_\infty = \displaystyle \lim_{p \rightarrow \infty} ||x||_p.$$

->En la norma $\mathcal{l}_2$ o Euclidiana $||x||_2$ tenemos una desigualdad muy importante, la desigualdad de **Cauchy-Schwartz**:

$$|x^Ty| \leq ||x||_2||y||_2 \forall x,y \in \mathbb{R}^n$$

la cual relaciona el producto interno estándar para $x,y \in \mathbb{R}^n$: $<x,y> = x^Ty = \displaystyle \sum_{i=1}^nx_iy_i$ con la norma $\mathcal{l}_2$ de $x$ y la norma $\mathcal{l}_2$ de $y$. Además se utiliza lo anterior para definir el ángulo (sin signo) entre $x,y$: 

$$\measuredangle x,y = \cos ^{-1}\left(\frac{x^Ty}{||x||_2||y||_2} \right )$$

para $\cos^{-1}(u) \in [0,\pi]$ y se nombra a $x,y$ ortogonales si $x^Ty=0$. Obsérvese que $||x||_2 = \sqrt{x^Tx}$.

* También se utilizan matrices* para definir normas

*Matriz: arreglo $2$-dimensional de datos o $2$ arreglo de orden $2$. Se utiliza la notación $A \in \mathbb{R}^{m\times n}$ para denotar:


$$A = \left[\begin{array}{cccc}
a_{11} &a_{12}&\dots&a_{1n}\\
a_{21} &a_{22}&\dots&a_{2n}\\
\vdots &\vdots& \vdots&\vdots\\
a_{n1} &a_{n2}&\dots&a_{nn}\\
\vdots &\vdots& \vdots&\vdots\\
a_{m-11} &a_{m-12}&\dots&a_{m-1n}\\
a_{m1} &a_{m2}&\dots&a_{mm}
\end{array}
\right] 
$$


*$a_{ij} \mathbb{R} \forall i=1,\dots,m, j=1,\dots,n$.

*$A=(a_1,\dots a_n), a_j \in \mathbb{R}^m (=\mathbb{R}^{m\times1}) \forall j=1,\dots,n$.

*$A=\left ( \begin{array}{c}
a_1^T\\
\vdots\\
a_m^T
\end{array} \right ), a_i \in \mathbb{R}^n (=\mathbb{R}^{n\times1}) \forall i=1,\dots,m$.

**Preguntas de comprehensión**

1)¿Qué factores influyen en la falta de exactitud de un cálculo?

2)Menciona $5$ propiedades que un conjunto debe cumplir para que sea considerado un espacio vectorial.

3)Menciona las propiedades que debe cumplir una función para que se considere una norma.

4)¿Qué es una norma matricial inducida?, ¿qué mide una norma matricial inducida?

5)¿La norma de Frobenius, es una norma matricial inducida?

6)¿A qué son iguales $\text{sup}(\emptyset)$, $\text{inf}(\emptyset)$ ? (el conjunto $\emptyset$ es el conjunto vacío)

7)Si f es un problema mal condicionado, ¿a qué nos referimos? Da ejemplos de problemas bien y mal condicionados.

8)Si f es un problema que resolvemos con un algoritmo g, ¿qué significa:

    a. que g sea estable?
    
    b. que g sea estable hacia atrás?
    
    c. que g sea inestable?
    
9)¿Qué ventaja(s) se tiene(n) al calcular un error hacia atrás vs calcular un error hacia delante?

